## Import libraries

### Run on colab

In [ ]:
%cd /content
!git clone https://github.com/Giovannicus/GreenTech-AI.git
%cd GourmetAI
!git checkout main

/content
Cloning into 'GourmetAI'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171 (from 1)
Receiving objects: 100% (171/171), 159.31 MiB | 10.39 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/GourmetAI
Updating files: 100% (75/75), done.
Branch 'colab_test' set up to track remote branch 'colab_test' from 'origin'.
Switched to a new branch 'colab_test'


### install libraries

In [ ]:
!pip install -r requirements.txt

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import os
import torch.optim as optim
import torchvision
from torchvision.models import vgg16, VGG16_Weights
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from src import *
from torchinfo import summary
import logging
import time

c:\Users\gcusumano\ProfAI\GreenTech-AI\.venv\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Data preprocessing

### extract images zip

In [3]:
# avoid time consuming importing dataset on colab, unzipping is faster
download_and_extract("https://proai-datasets.s3.eu-west-3.amazonaws.com/progetto-finale-flowes.tar.gz")

Extracting files...
Files extracted in: .


### transform and get dataloaders

In [2]:
# transform images to get some augmentation
class Transforms:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, *args, **kwargs):
        return self.transforms(image=np.array(img))['image']

transform = A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(),
        A.Rotate(limit=10),
        A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ToTensorV2(),
      ])

transform2 = A.Compose([
        A.Resize(224, 224),
        A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ToTensorV2(),
      ])

In [3]:
trainset = torchvision.datasets.ImageFolder(root='dataset/train', transform=Transforms(transform))
valset = torchvision.datasets.ImageFolder(root='dataset/valid', transform=Transforms(transform2))
testset = torchvision.datasets.ImageFolder(root='dataset/test',transform=Transforms(transform2))

In [4]:
len(trainset), len(valset), len(testset)

(2550, 728, 364)

In [5]:
def fix_path(path_tuple):
    path, label = path_tuple
    fixed_path = path.replace('\\', '/')
    return (fixed_path, label)

In [6]:
trainset.imgs = [fix_path(item) for item in trainset.imgs]
valset.imgs = [fix_path(item) for item in valset.imgs]
testset.imgs = [fix_path(item) for item in testset.imgs]

In [7]:
from torch.utils.data import Subset
batch_size=32

# stratified kfold to avoid unbalanced dataset
train_indices = get_balanced_indices(trainset, len(trainset))
val_indices = get_balanced_indices(valset, len(valset))
test_indices = get_balanced_indices(testset, len(testset))

balanced_trainset = Subset(trainset, train_indices)
balanced_valset = Subset(valset, val_indices)
balanced_testset = Subset(testset, test_indices)

trainloader = torch.utils.data.DataLoader(balanced_trainset,batch_size=batch_size,shuffle=True) 
valloader = torch.utils.data.DataLoader(balanced_valset,batch_size=batch_size,shuffle=False)
testloader = torch.utils.data.DataLoader(balanced_testset,batch_size=batch_size,shuffle=False)


### check distribution 

In [8]:
label_counts = torch.zeros(len(trainset.classes))
for _, labels in trainloader.dataset.dataset.imgs:
    label_counts[labels] += 1

In [9]:
# interactive pieplot
fig = plot_class_distribution(
    label_counts,
    class_names=trainset.classes,
    title="Dataset Distribution",
    paper_bgcolor="rgba(255255,255,.1)",
    plot_bgcolor="rgba(0,0,0,0)",
    t=10,b=10,l=10,r=100,
    height=600,width=600,
    legend_x_anchor="center",
    legend_y_anchor="middle",
    legend_orientation = "v",
    legend_x = -.3,
    legend_y = 1
)
fig.show()

## Transfer Learning

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"

vgg = vgg16(weights = VGG16_Weights.IMAGENET1K_V1).to(device)
for param in vgg.parameters():
    param.requires_grad = False

class VGGClassifier(nn.Module):
    def __init__(self, vgg_backbone, in_features, num_classes):
        super().__init__()
        self.features = vgg_backbone.features
        self.pooling = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(in_features, num_classes)
    def forward(self, x):
        x = self.features(x)
        x = self.pooling(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
    
vgg = VGGClassifier(vgg,512,len(trainset.classes)).to(device)
summary(vgg, input_size=(1, 3, 224, 224), device=device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 92.8MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## Training

### set train test

In [ ]:
def train(net, dataloader, criterion, optimizer,device):
    """ trains an epoch and returns metrics"""
    net.train()
    n_batches = len(dataloader)
    total_loss = 0
    total_precision = 0
    for inputs, targets in dataloader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        output = net(inputs)

        net.zero_grad()

        loss = criterion(output, targets)

        loss.backward()
        optimizer.step()

        _, preds = torch.max(output.data, 1)
        precision = get_precision(preds, targets)

        total_loss += loss.item()
        total_precision += precision

    mean_loss = total_loss / n_batches
    mean_precision = total_precision / n_batches
    return mean_loss, mean_precision

def test(net, test_loader, criterion,device):

    net.eval()
    test_loss = 0
    test_precision = 0
    n_batches = len(test_loader)

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)

            output = net(data)
            test_loss += criterion(output, target).item()

            _, preds = torch.max(output.data, 1)
            batch_precision = get_precision(preds, target)
            test_precision += batch_precision

    test_loss /= n_batches
    test_precision /= n_batches

    print(f"\nFinal validation metrics:")
    print(f"Loss: {test_loss:.4f}")
    print(f"Precision: {test_precision:.4f}")

    return test_loss, test_precision

def get_predictions(model_output):
    val, idx = torch.max(model_output, dim=1)
    return idx

def get_precision(preds: torch.Tensor, targets: torch.Tensor, threshold: float = 0.5) -> float:
    """ returns mean precision ignoring nan or non-involved classes"""
    device = targets.device
    preds = preds.to(device)

    preds = manual_one_hot(preds,14)
    targets = manual_one_hot(targets,14)
    
    precisions = []

    
    num_classes = preds.shape[1]
    for class_idx in range(num_classes):

        class_preds = preds[:, class_idx]
        class_targets = targets[:, class_idx]

        
        true_positives = (class_preds * class_targets).sum()
        false_positives = (class_preds * (1 - class_targets)).sum()
        total_predicted = true_positives+false_positives

        
        if total_predicted > 0:
            class_precision = true_positives / total_predicted
            precisions.append(class_precision)

        else:
            class_precision = torch.nan
            precisions.append(class_precision)

    precision_tensor = torch.tensor(precisions)
    mean_precision = precision_tensor.nanmean().item()

    return mean_precision


def adjust_learning_rate(lr, decay, optimizer, cur_epoch, n_epochs):
    """Sets the learning rate to the initially
        configured `lr` decayed by `decay` every `n_epochs`"""
    new_lr = lr * (decay ** (cur_epoch // n_epochs))
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_uniform(m.weight)
        m.bias.data.zero_()

def save_weights(model, weights_dir, epoch):
    weights_fname = 'weights-%d.pth' % (epoch)
    weights_fpath = os.path.join(weights_dir, weights_fname)
    torch.save({'state_dict': model.state_dict()}, weights_fpath)

def load_weights(model, fpath):
    state = torch.load(fpath)
    model.load_state_dict(state['state_dict'])

def manual_one_hot(tensor, num_classes=None):
    if num_classes is None:
        num_classes = tensor.max() + 1
    one_hot = torch.zeros((tensor.size(0), num_classes),device=device)
    one_hot.scatter_(1, tensor.unsqueeze(1), 1)
    return one_hot

### new experiment

In [ ]:
N_EPOCHS = 30
MAX_PATIENCE = 5
LEARNING_RATE = 1e-3
LR_DECAY = 0.99
DECAY_LR_EVERY_N_EPOCHS = 2
EXPERIMENT_NAME = 'exp1'

model = vgg.to(device)
criterion = nn.CrossEntropyLoss().to(device)
logger = get_logger(ch_log_level=logging.INFO, fh_log_level=logging.INFO)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print('  + Number of params: {}'.format(sum([p.data.nelement() for p in model.parameters()])))
exp = Experiment(EXPERIMENT_NAME, "exps", logger)

exp.init()
monitor = TrainingMonitor()

for epoch in range(exp.epoch, exp.epoch+N_EPOCHS):
    since = time.time()

    trn_loss, trn_precision = train(model, trainloader, criterion, optimizer,device)
    logger.info('Epoch {:d}: Train - Loss: {:.4f}\tPrecision: {:.4f}'.format(
        epoch, trn_loss, trn_precision))
    time_elapsed = time.time() - since
    logger.info('Train Time {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    val_loss, val_precision = test(model, valloader, criterion,device)
    logger.info('Val - Loss: {:.4f}, Precision: {:.4f}'.format(
        val_loss, val_precision))
    time_elapsed = time.time() - since
    logger.info('Total Time {:.0f}m {:.0f}s\n'.format(
        time_elapsed // 60, time_elapsed % 60))

    monitor.update(
        epoch=epoch,
        train_loss=trn_loss,
        val_loss=val_loss,
        train_precision=trn_precision,
        val_precision=val_precision
    )

    exp.save_history('train', trn_loss, trn_precision)
    exp.save_history('val', val_loss, val_precision)

    exp.save_weights(model, trn_loss, val_loss, trn_precision, val_precision)
    exp.save_optimizer(optimizer, val_loss)

    if (epoch - exp.best_val_loss_epoch) > MAX_PATIENCE:
        logger.info(f"Early Stopping at epoch {epoch} since no better loss found since epoch {exp.best_val_loss_epoch}")
        break

    adjust_learning_rate(LEARNING_RATE, LR_DECAY, optimizer,
                         epoch, DECAY_LR_EVERY_N_EPOCHS)

    exp.epoch += 1

try:
    monitor.save(os.path.join(exp.history_dir, 'final_training_progress.png'))
    monitor.stop()
except:
    pass

In [ ]:
### run if already have exp and avoid training
import zipfile
with zipfile.ZipFile("exp1.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

### plot training history

In [ ]:
fig = plot_training_history('exps/exp1')
fig

## Final test

In [ ]:
experiment = Experiment(name="exp1", root="exps")

# load model best hyperparams
model, optimizer = experiment.resume(
    model=model,  
    optim=optimizer,  
    weights_fpath=experiment.best_weights_path,  
    optim_path=experiment.best_optimizer_path  
)

<ipython-input-10-c7cefe42d424>:154: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

<ipython-input-10-c7cefe42d424>:175: FutureWarning:

You are using `torch.load` with `weig

### set final test and confusion matrix

In [ ]:
def get_confusion_matrix(preds: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    """
    Calculate confusion matrix given predictions and targets.

    Args:
        preds: torch.Tensor - Model predictions (already processed with argmax if necessary)
        targets: torch.Tensor - Ground truth targets

    Returns:
        confusion_matrix: torch.Tensor - Confusion matrix of shape (num_classes x num_classes)
    """
    confusion_matrix = torch.zeros(14, 14)
    for t, p in zip(targets.view(-1), preds.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1
    return confusion_matrix

def calculate_metrics(confusion_matrix: torch.Tensor) -> dict:
    """
    Calculate various metrics from the confusion matrix using only PyTorch operations.

    Args:
        confusion_matrix: torch.Tensor - Confusion matrix of shape (num_classes x num_classes)

    Returns:
        dict: Dictionary containing various metrics
    """
    # Per-class metrics
    class_precision = confusion_matrix.diag() / confusion_matrix.sum(dim=1).clamp(min=1e-10)
    class_recall = confusion_matrix.diag() / confusion_matrix.sum(dim=0).clamp(min=1e-10)

    # Calculate F1 score
    class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall + 1e-10)

    # Calculate macro averages
    metrics = {
        'macro_precision': float(torch.mean(class_precision)),
        'macro_recall': float(torch.mean(class_recall)),
        'macro_f1': float(torch.mean(class_f1)),
        'accuracy': float(confusion_matrix.diag().sum() / confusion_matrix.sum())
    }

    return metrics

def final_test(net, test_loader, criterion):
    """
    Perform final evaluation of the model on test data.

    Args:
        net: torch.nn.Module - The neural network model
        test_loader: torch.utils.data.DataLoader - Test data loader
        criterion: torch.nn.Module - Loss function

    Returns:
        tuple: (confusion_matrix, metrics_dict)
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.eval()
    test_loss = 0
    n_batches = len(test_loader)

    # Initialize tensors for all predictions and targets
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)

            output = net(data)
            test_loss += criterion(output, target).item()

            _, preds = torch.max(output.data, 1)

            # Store predictions and targets
            all_preds.append(preds.cpu())
            all_targets.append(target.cpu())

    # Concatenate all batches
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)

    # Calculate final metrics
    test_loss /= n_batches
    confusion_matrix = get_confusion_matrix(all_preds, all_targets)
    metrics = calculate_metrics(confusion_matrix)

    # Print results
    print(f"\nFinal validation metrics:")
    print(f"Loss: {test_loss:.4f}")
    print("\nMetrics from confusion matrix:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")

    return confusion_matrix, metrics


### visualize confusion matrix

In [ ]:
confusion_matrix, metrics = final_test(model, testloader, criterion)
class_names = trainset.classes
plot_confusion_matrix(confusion_matrix,class_names)

## Conclusion

0.8 metrics score indicates that the model has quite well learned to distinguish between most food categories. However, model might fail predicting eye-looking similar foodslike taco and taquito. Possible improvements:
-   Guideline for taking food picks:
    Position food items to highlight their distinctive characteristics (e.g., size, surface patterns) and maintain consistent feature visibility.
    Use neutral backgrounds to minimize distractions and emphasize food characteristics.
    This helps model focusing on essential features.

For the second point about handling similar foods:
-   Hierical classification:
    Primary classifier: Categorizes foods into broad categories (e.g., burrito-like, pasta, ecc..).
    Specialized sub-classifiers: Dedicated models trained specifically to distinguish between similar items within each category.
    This reduces the complexity of single models avoiding overfitting when there are tens of classes.
